In [1]:
import numpy as np
import astropy.units as u
from astropy.coordinates import SkyCoord, ICRS, Galactic
import uncertainties as unc
import table_utils as tu
import glob

import pint.derived_quantities as dq
#See num2tex (https://github.com/AndrewChap/num2tex)
# > pip install num2tex
from num2tex import num2tex

In [2]:
psrs = [
    "J0032+6946",
    "J0141+6303",
    "J0214+5222",
    "J0415+6111",
    "J0636+5128",
    "J0957-0619",
    "J1239+3239",
    "J1327+3423",
    "J1434+7257",
    "J1505-2524",
    "J1530-2114",
    "J1816+4510",
    "J1913+3732",
    "J1929+6630",
    "J1930+6205",
    "J2104+2830",
    "J2115+6702",
    "J2145+2158",
    "J2210+5712",
    "J2326+6243",
    "J2354-2250"
]

In [3]:
# Auto-populate rotational/timing params table
pars = sorted(glob.glob(f"data/*_fiore+23.par"))

f0ve = {}
f1ve = {}

for par in pars:
    with open(par, 'r') as infile:
        f2 = "--"
        for l in infile.readlines():
            if l.startswith("PSRJ"):
                psr     = l.split()[1]
                psr_tex = psr.replace('-','$-$')
            elif l.startswith("F0"):
                F0      = float(l.split()[1])
                F0_err  = float(l.split()[3])
                f0      = tu.ufve(F0, F0_err)
                if len(f0.split(".")[0]) == 1:
                    f0 = f"\dphn{f0}"
                elif len(f0.split(".")[0]) == 2:
                    f0 = f"\phn{f0}"
                f0ve[psr] = [F0*u.Hz, F0_err*u.Hz]
            elif l.startswith("F1"):
                F1      = float(l.split()[1])
                F1_err  = float(l.split()[3])
                f1      = tu.ufve(F1, F1_err).replace("^","$^").replace("}","}$")
                f1ve[psr] = [F1*u.Hz*u.s**-1, F1_err*u.Hz*u.s**-1]
            elif l.startswith("F2"):
                F2      = float(l.split()[1])
                F2_err  = float(l.split()[3])
                f2      = tu.ufve(F2, F2_err).replace("^","$^").replace("}","}$")
            elif l.startswith("PEPOCH"):
                epoch   = l.split()[1]
            elif l.startswith("START"):
                start   = float(l.split()[1])
            elif l.startswith("FINISH"):
                finish  = float(l.split()[1])
            elif l.startswith("TRES"):
                rms_us  = round(float(l.split()[1]), 1)
                if len(str(rms_us).split(".")[0]) == 1:
                    rms_us = f"\\tphn{rms_us}"
                elif len(str(rms_us).split(".")[0]) == 2:
                    rms_us = f"\dphn{rms_us}"
                elif len(str(rms_us).split(".")[0]) == 3:
                    rms_us = f"\phn{rms_us}"
            elif l.startswith("NTOA"):
                ntoa    = l.split()[1]
                if len(ntoa) == 2:
                    ntoa = f"\dphn{ntoa}"
                elif len(ntoa) == 3:
                    ntoa = f"\phn{ntoa}"
            elif l.startswith("CHI2R"):
                chi2r   = float(l.split()[1])
                efac    = round(np.sqrt(chi2r), 2)
            elif l.startswith("EFAC"):
                efac    = round(float(l.split()[3]),2)
            else:
                pass
        span = f"{int(start)}--{int(finish)}"
    # '\' is a special python character, but prefixing with '\' turns it into a normal character print('\\') -> '\'
    #out_str = f"{psr_tex} & {f0} & {f1} & {f2} & {epoch} & {span} & {rms_us:.1f} & {ntoa} & {chi2r:.2f} \\\\"
    if psr=="J0032+6946":
        rms_us = "19.1"
    elif psr=="J0214+5222":
        rms_us = "77.8"
    out_str = f"{psr_tex} & {f0} & {f1} & {epoch} & {span} & {rms_us} & {ntoa} & {efac} \\\\"
    print(out_str.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("$-$-","--"))

J0032+6946 & \phn27.171119572492(3) & $-$2.65006(3)$\times$10$^{-15}$ & 56736.0000000000000000 & 55169--58303 & 19.1 & 1430 & 1.08 \\
J0141+6303 & \phn21.42232445491(4) & $-$7.65(2)$\times$10$^{-16}$ & 57431 & 57072--57789 & \dphn75.1 & \phn122 & 1.03 \\
J0214+5222 & \phn40.691271761863(5) & $-$4.9002(7)$\times$10$^{-16}$ & 56974.0000000000000000 & 55353--58594 & 77.8 & \phn951 & 1.05 \\
J0415+6111 & \dphn2.27174933348(6) & $-$2.8(4)$\times$10$^{-16}$ & 57234 & 57071--57397 & \phn609.4 & \dphn38 & 1.04 \\
J0636+5128 & 348.55923172059(1) & $-$4.262(8)$\times$10$^{-16}$ & 56712 & 56027--57397 & \tphn1.9 & 1403 & 1.17 \\
J0957$-$0619 & \dphn0.58014346794(2) & $-$5(1)$\times$10$^{-17}$ & 57220 & 57071--57369 & \phn767.0 & \dphn67 & 1.07 \\
J1239+3239 & 212.71645129924(2) & $-$1.752(5)$\times$10$^{-16}$ & 57733 & 56054--59412 & \dphn21.5 & \phn283 & 1.13 \\
J1327+3423 & \phn24.089008071282(2) & $-$7.514(3)$\times$10$^{-17}$ & 58067 & 57079--59055 & \tphn2.7 & 1575 & 1.18 \\
J1434+7257 & \ph

In [4]:
# Auto-populate derived common properties table

for psr in psrs:
    par_path = f"data/{psr}_fiore+23.par"
    psr_tex = psr.replace('-','$-$')
    
    f0,f0err = f0ve[psr]
    f1,f1err = f1ve[psr]
            
    # Calculate derived quantities with PINT
    p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
    p_tex = tu.ufve(p.value,perr.value)
    pd_tex = tu.ufve(pd.value,pderr.value)
    age = f"{num2tex(dq.pulsar_age(f0,f1).value):.1e}" # yr
    bsurf = f"{num2tex(dq.pulsar_B(f0,f1).value):.1e}" # G
    edot = f"{num2tex(dq.pulsar_edot(f0,f1).value):.1e}" # erg/s
    
    out_str = f"{psr_tex} & {p_tex} & {pd_tex} & {age} & {bsurf} & {edot} \\\\"
    
    print(out_str.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0032+6946 & 0.036803783419083(3) & 3.58955(4)$\times$10$^{-18}$ & 1.6$\times$10$^{8}$ & 1.2$\times$10$^{10}$ & 2.8$\times$10$^{33}$ \\
J0141+6303 & 0.04668027515432(9) & 1.666(4)$\times$10$^{-18}$ & 4.4$\times$10$^{8}$ & 8.9$\times$10$^{9}$ & 6.5$\times$10$^{32}$ \\
J0214+5222 & 0.024575294816350(3) & 2.9594(4)$\times$10$^{-19}$ & 1.3$\times$10$^{9}$ & 2.7$\times$10$^{9}$ & 7.9$\times$10$^{32}$ \\
J0415+6111 & 0.44018941054(1) & 5.5(8)$\times$10$^{-17}$ & 1.3$\times$10$^{8}$ & 1.6$\times$10$^{11}$ & 2.5$\times$10$^{31}$ \\
J0636+5128 & 0.00286895284644653(9) & 3.508(7)$\times$10$^{-21}$ & 1.3$\times$10$^{10}$ & 1.0$\times$10$^{8}$ & 5.9$\times$10$^{33}$ \\
J0957$-$0619 & 1.72371155631(5) & 1.5(3)$\times$10$^{-16}$ & 1.8$\times$10$^{8}$ & 5.1$\times$10$^{11}$ & 1.2$\times$10$^{30}$ \\
J1239+3239 & 0.0047010938453146(4) & 3.87(1)$\times$10$^{-21}$ & 1.9$\times$10$^{10}$ & 1.4$\times$10$^{8}$ & 1.5$\times$10$^{33}$ \\
J1327+3423 & 0.041512709740513(3) & 1.2948(5)$\times$10$^{-19}$ & 5.1$

In [5]:
# Auto-populate coordinates/DM table
rave = {}
decve = {}
dmve = {}

for psr in psrs:    
    par_path = f"data/{psr}_fiore+23.par"
    psr_tex = psr.replace('-','$-$')
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("RAJ"):
                ra_str = l.split()[1]
                ra_err = float(l.split()[3])
                rave[psr] = [ra_str,ra_err]
            elif l.startswith("DECJ"):
                dec_str = l.split()[1].strip("+")
                dec_err = float(l.split()[3])
                decve[psr] = [dec_str,dec_err]
            elif l.startswith("DM "):
                dm_val = float(l.split()[1])
                try:
                    dm_err = float(l.split()[3])
                    dmve[psr] = [dm_val,dm_err]
                    dm_tex = tu.ufve(dm_val,dm_err)
                except IndexError:
                    dmve[psr] = [dm_val,None]
                    dm_tex = f"{dm_val:.4f}"
    
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))

    ra = tu.format_ra(eqcoord,ra_err)
    dec = tu.format_dec(eqcoord,dec_err)
    
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm_val)
    if dmdist_ne < 10.:
        dmdist_ne = f"\phn{dmdist_ne:.1f}"
    else:
        dmdist_ne = f"{dmdist_ne:.1f}"
    if dmdist_ymw < 10.:
        dmdist_ymw = f"\phn{dmdist_ymw:.1f}"
    else:
        dmdist_ymw = f"{dmdist_ymw:.1f}"
    gl = f"{gcoord.l.deg:.2f}"
    gb = f"{gcoord.b.deg:.2f}".replace('-','$-$')
    print(f"{psr_tex} & {ra} & {dec} & {dm_tex} & {gl} & {gb} & {dmdist_ne} & {dmdist_ymw} \\\\")#.replace("^","$^").replace("}","}$"))

J0032+6946 & $00^{\rm h}\, 32^{\rm m}\, 41\, \fs2477(3)$ & $+69\arcdeg\, 46\arcmin\, 28\, \farcs047(2)$ & 79.9988(2) & 121.30 & 6.96 & \phn2.8 & \phn2.3 \\
J0141+6303 & $01^{\rm h}\, 41^{\rm m}\, 45\, \fs761(1)$ & $+63\arcdeg\, 03\arcmin\, 49\, \farcs445(9)$ & 272.762(2) & 128.60 & 0.75 & 44.3 & \phn8.8 \\
J0214+5222 & $02^{\rm h}\, 14^{\rm m}\, 55\, \fs2746(2)$ & $+52\arcdeg\, 22\arcmin\, 40\, \farcs907(3)$ & 22.0371(3) & 135.63 & $-$8.42 & \phn1.0 & \phn1.2 \\
J0415+6111 & $04^{\rm h}\, 15^{\rm m}\, 51\, \fs63(5)$ & $+61\arcdeg\, 11\arcmin\, 51\, \farcs8(3)$ & 70.8(1) & 145.15 & 7.49 & \phn2.3 & \phn1.8 \\
J0636+5128 & $06^{\rm h}\, 36^{\rm m}\, 04\, \fs84705(3)$ & $+51\arcdeg\, 28\arcmin\, 59\, \farcs9658(6)$ & 11.1075 & 163.91 & 18.64 & \phn0.5 & \phn0.2 \\
J0957$-$0619 & $09^{\rm h}\, 57^{\rm m}\, 08\, \fs12(2)$ & $-06\arcdeg\, 19\arcmin\, 37\, \farcs5(9)$ & 27.3(1) & 244.83 & 36.20 & \phn1.2 & \phn2.5 \\
J1239+3239 & $12^{\rm h}\, 39^{\rm m}\, 27\, \fs3140(1)$ & $+32\arcdeg\, 39\

In [6]:
# Auto-populate proper motion, intrinsic P-dot table
psr_names = ['J0214+5222','J0636+5128','J1327+3423','J1434+7257','J1816+4510']
for psr in psr_names:
    par_path = f"data/{psr}_fiore+23.par" 
    
    psr_tex = psr.replace('-','$-$')
    ra_str, ra_err   = rave[psr]
    dec_str, dec_err = decve[psr]
    dm, dm_err = dmve[psr]
    has_pmdec = False
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
            if l.startswith("PMRA"):
                pmra_val = float(l.split()[1])
                pmra_err = float(l.split()[3])
                pmra = f"{tu.ufve(pmra_val,pmra_err)}".replace("-","$-$")
            elif l.startswith("PMDEC"):
                has_pmdec = True
                pmdec_val = float(l.split()[1])
                try:
                    pmdec_err = float(l.split()[3])
                except IndexError:
                    has_pmdec = False
            else:
                pass
            
    eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))
    gcoord = eqcoord.transform_to(Galactic)
    dmdist_ne, dmdist_ymw = tu.get_dmdists(gcoord,dm)
    
    if has_pmdec:
        pmdec = f"{tu.ufve(pmdec_val,pmdec_err)}".replace("-","$-$")
        pmtot,pmtoterr = tu.PMtot_err(pmra_val*u.mas/u.yr,pmra_err*u.mas/u.yr,pmdec_val*u.mas/u.yr,pmdec_err*u.mas/u.yr)
    else:
        pmdec = "\ldots"
        pmtot,pmtoterr = tu.PMtot_err(pmra_val*u.mas/u.yr,pmra_err*u.mas/u.yr,0.0*u.mas/u.yr,0.0*u.mas/u.yr)

    for ii,dmdist in enumerate([dmdist_ne,dmdist_ymw]):
        dd,de = (dmdist,dmdist*0.3)*u.kpc
        dist = tu.ufve(dd.value,de.value)
        vt,vterr = tu.Vtrans_err(pmtot,pmtoterr,dd,de)
        vt_str = tu.ufve(vt.value,vterr.value)

        f0,f0err = f0ve[psr]
        f1,f1err = f1ve[psr]
        p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
        pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
        pds = f"{pdshk*1e21:.2f}"
        pdgal = tu.pd_gal(p,gcoord,dd).decompose()
        pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

        # Am I doing the math here correctly?
        pdint = pd-(pdshk+pdgal)
        pdi = f"{pdint.value*1e19:.2f}"

        # Re-derive age, bsurf, edot with intrinsic P-dot
        f1 = -1.0*pdint/p**2
        age = f"{dq.pulsar_age(f0,f1).value*1e-9:.1f}" # Gyr
        bsurf = f"{dq.pulsar_B(f0,f1).value*1e-9:.1f}" # 10^9 G
        edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.1f}" # 10^33 erg/s

        if ii == 0:
            print(f"{psr_tex} & {pmra} & {pmdec} & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\".replace(" \\times ","$\\times$").replace("^","$^").replace("}","}$"))
        elif ii == 1:
            print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\".replace(" \\times ","$\\times$").replace("^","$^").replace("}","}$"))

J0214+5222 & 9(1) & 2(2) & 1.0(3) & 4(1)$\times$10$^{1}$ & $-$0.04 & 5.14 & 2.91 & 2.7 & 1.3 & 0.8 \\
 & & & 1.2(3) & 5(2)$\times$10$^{1}$ & $-$0.02 & 5.79 & 2.90 & 2.7 & 1.3 & 0.8 \\
J0636+5128 & 1.1(4) & $-$4.4(7) & 0.5(1) & 11(4) & 0.03 & 0.07 & 0.03 & 0.1 & 13.3 & 5.7 \\
 & & & 0.21(6) & 5(2) & 0.01 & 0.03 & 0.03 & 0.1 & 13.1 & 5.8 \\
J1327+3423 & $-$8.2(2) & 4.3(4) & 0.5(1) & 21(6) & $-$5.84 & 4.07 & 1.31 & 2.4 & 5.0 & 0.1 \\
 & & & 0.3(1) & 15(5) & $-$4.92 & 2.94 & 1.31 & 2.4 & 5.0 & 0.1 \\
J1434+7257 & $-$4.5(9) & $-$7.6(6) & 0.7(2) & 30(9) & $-$4.96 & 5.63 & 5.48 & 4.8 & 1.2 & 0.3 \\
 & & & 1.0(3) & 4(1)$\times$10$^{1}$ & $-$5.87 & 7.69 & 5.47 & 4.8 & 1.2 & 0.3 \\
J1816+4510 & 2(1) & $-$3(1) & 2.4(7) & 4(2)$\times$10$^{1}$ & $-$0.84 & 0.22 & 0.44 & 0.4 & 1.2 & 53.0 \\
 & & & 4(1) & 7(3)$\times$10$^{1}$ & $-$1.42 & 0.39 & 0.44 & 0.4 & 1.1 & 53.5 \\


In [7]:
#### A row for the above table using the Lutz-Kelker-corrected parallax distance for PSR J1327+3423 ####
par_path = f"data/J1327+3423_fiore+23.par"
ra_str, ra_err   = rave[psr]
dec_str, dec_err = decve[psr]
dm, dm_err = dmve[psr]
with open(par_path, 'r') as infile:
    for l in infile.readlines():
        if l.startswith("PMRA"):
            pmra_val = float(l.split()[1])
            pmra_err = float(l.split()[3])
            pmra = f"{tu.ufve(pmra_val,pmra_err)}".replace("-","$-$")
        elif l.startswith("PMDEC"):
            try:
                pmdec_val = float(l.split()[1])
                pmdec_err = float(l.split()[3])
            except IndexError:
                pmdec = "\ldots"
            pmdec = f"{tu.ufve(pmdec_val,pmdec_err)}".replace("-","$-$")
        else:
            pass
        
eqcoord = SkyCoord(ra_str,dec_str,frame=ICRS,unit=(u.hourangle, u.deg))
gcoord = eqcoord.transform_to(Galactic)
pmtot,pmtoterr = tu.PMtot_err(pmra_val*u.mas/u.yr,pmra_err*u.mas/u.yr,pmdec_val*u.mas/u.yr,pmdec_err*u.mas/u.yr)

pxdist = 0.9
dd,de = (pxdist,0.6)*u.kpc
dist = "0.9(0.6)"
vt,vterr = tu.Vtrans_err(pmtot,pmtoterr,dd,de)
vt_str = tu.ufve(vt.value,vterr.value)

f0,f0err = f0ve[psr]
f1,f1err = f1ve[psr]
p,perr,pd,pderr = dq.pferrs(f0,f0err,f1,f1err)
pdshk = dq.shklovskii_factor(pmtot,dd)*p.decompose()
pds = f"{pdshk*1e21:.2f}"
pdgal = tu.pd_gal(p,gcoord,dd).decompose()
pdg = f"{pdgal*1e21:.2f}".replace("-","$-$")

# Am I doing the math here correctly?
pdint = pd-(pdshk+pdgal)
pdi = f"{pdint.value*1e19:.2f}"

# Re-derive age, bsurf, edot with intrinsic P-dot
f1 = -1.0*pdint/p**2
age = f"{dq.pulsar_age(f0,f1).value*1e-9:.2f}" # Gyr
bsurf = f"{dq.pulsar_B(f0,f1).value*1e-9:.2f}" # 10^9 G
edot = f"{dq.pulsar_edot(f0,f1).value*1e-33:.2f}" # 10^33 erg/s

print(f" & & & {dist} & {vt_str} & {pdg} & {pds} & {pdi} & {bsurf} & {age} & {edot} \\\\".replace(" \\times ","$\\times$").replace("^","$^").replace("}","}$"))

 & & & 0.9(0.6) & 4(3)$\times$10$^{1}$ & $-$0.36 & 0.59 & 0.43 & 0.37 & 1.18 & 51.96 \\


In [8]:
# Auto-populate binaries table
max_masses = []
med_masses = []

binary_psrs = ['J0032+6946','J0214+5222','J0636+5128','J1239+3239','J1816+4510']
for bp in binary_psrs:
    par_path = f"data/{bp}_fiore+23.par"
    
    fb1_tex = '\ldots'
    fb2_tex = '\ldots'
    xdot_tex = '\ldots'
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        #pbdot_tex = "\ldots"
        for l in infile.readlines():
            if l.startswith("A1 "):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
#             elif l.startswith("PB "):
#                 pb_val = float(l.split()[1])
#                 pb_err = float(l.split()[3])
#                 pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("FB0"):
                fb_val = float(l.split()[1])
                fb_err = float(l.split()[3])
                fb_tex = tu.ufve(fb_val,fb_err)
#             elif l.startswith("PBDOT"):
#                 pbdot_val = float(l.split()[1])
#                 pbdot_err = float(l.split()[3])
#                 pbdot_tex = tu.ufve(pbdot_val,pbdot_err)
            elif l.startswith("TASC"):
                tasc_val = float(l.split()[1])
                tasc_err = float(l.split()[3])
                tasc_tex = tu.ufve(tasc_val,tasc_err)
            elif l.startswith("EPS1 "):
                eps1_val = float(l.split()[1])
                eps1_err = float(l.split()[3])
                eps1_tex = tu.ufve(eps1_val,eps1_err)
            elif l.startswith("EPS2 "):
                eps2_val = float(l.split()[1])
                eps2_err = float(l.split()[3])
                eps2_tex = tu.ufve(eps2_val,eps2_err)
            elif l.startswith("FB1"):
                fb1_val = float(l.split()[1])
                fb1_err = float(l.split()[3])
                fb1_tex = tu.ufve(fb1_val, fb1_err)
            elif l.startswith("FB2"):
                fb2_val = float(l.split()[1])
                fb2_err = float(l.split()[3])
                fb2_tex = tu.ufve(fb2_val, fb2_err)
            elif l.startswith("XDOT") or l.startswith("A1DOT"):
                xdot_val = float(l.split()[1])
                try:
                    xdot_err = float(l.split()[3])
                    xdot_tex = tu.ufve(xdot_val, xdot_err)
                except IndexError:
                    xdot_tex = "\ldots"
            else:
                pass
    
    # Derive mass function and Mc,min
    pb = (1./(fb_val*u.Hz)).to(u.day).value
    pb_err = (fb_err*u.Hz/(fb_val*u.Hz)**2).to(u.day).value
    pb_tex = tu.ufve(pb,pb_err)
    ecc = np.sqrt(eps1_val**2+eps2_val**2)
    ecc_err = np.sqrt((eps1_val*eps1_err)**2+(eps2_val*eps2_err)**2)/ecc
    ecc_tex = tu.ufve(ecc,ecc_err)
    om = np.arctan2(eps1_val,eps2_val)*180./np.pi
    if om < 0.:
        om += 360.
    om_err = (180./np.pi)*np.sqrt((eps2_val*eps1_err)**2+(eps1_val*eps2_err)**2)/ecc**2
    if om_err < 1.0:
        om_tex = tu.ufve(om,om_err)
    else:
        om_tex = f"{int(round(om,0))}({int(round(om_err,0))})"
    t0 = tasc_val + pb*om/360.
    t0_err = np.sqrt(tasc_err**2+(pb_err*om/360.)**2+(pb*om_err/360.)**2)
    t0_tex = tu.ufve(t0,t0_err)
    fm = dq.mass_funct(pb*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.2e}" # Msun
    mc = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_max = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=26.0*u.deg, mp=1.4 * u.solMass)
    mc_med = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=60.0*u.deg, mp=1.4 * u.solMass)
    max_masses.append(mc_max)
    med_masses.append(mc_med)
    mc_tex = f"{mc.value:.3f}" # Msun

    #outstr = f"{psr_tex} & {pb_tex} & {pbdot_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {eps2_tex} & {fm_tex} & {mc_tex} \\\\"
    outstr = f"{psr_tex} & {fb_tex} & {a1_tex} & {tasc_tex} & {eps1_tex} & {eps2_tex} & \
    {fb1_tex} & {fb2_tex} & {xdot_tex} & {pb_tex} & {t0_tex} & {om_tex} & {ecc_tex} & {fm_tex} & {mc_tex} \\\\"
    print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

J0032+6946 & 2.193632768(6)$\times$10$^{-8}$ & 178.674768(3) & 56399.134959(4) & 0.00028554(2) & $-$0.00044865(3) &     \ldots & \ldots & \ldots & 527.621316(2) & 56615.350(3) & 147.525(2) & 0.00053181(3) & 2.20$\times$10$^{-2}$ & 0.417 \\
J0214+5222 & 2.260385778(9)$\times$10$^{-8}$ & 174.565762(5) & 56339.115121(4) & $-$0.00271145(5) & $-$0.00458640(6) &     \ldots & \ldots & $-$2.1(6)$\times$10$^{-13}$ & 512.039767(2) & 56638.6460(8) & 210.5913(6) & 0.00532795(5) & 2.18$\times$10$^{-2}$ & 0.416 \\
J0636+5128 & 0.0001739119636(3) & 0.0089858(1) & 56711.9950666(4) & 1(2)$\times$10$^{-5}$ & $-$1(2)$\times$10$^{-5}$ &     $-$7.1(7)$\times$10$^{-20}$ & $-$2.4(8)$\times$10$^{-27}$ & \ldots & 0.0665513392(1) & 56712.02(2) & 145(106) & 1(2)$\times$10$^{-5}$ & 1.76$\times$10$^{-7}$ & 0.007 \\
J1239+3239 & 2.833032117(4)$\times$10$^{-6}$ & 2.371127(2) & 57730.0800015(9) & 5(2)$\times$10$^{-6}$ & 0(2)$\times$10$^{-6}$ &     \ldots & \ldots & \ldots & 4.085401647(6) & 57731.1(3) & 89(22) & 5(2)

In [9]:
# Auto-populate binaries table
max_masses = []
med_masses = []

param_names = ["$\\nu_\mathrm{b}$ (Hz)",
          "$a\sin i/c$ (s)",
          "$T_\mathrm{asc}$ (MJD)",
          "$e\sin\omega$",
          "$e\cos\omega$",
          "$\dot{\\nu}_\mathrm{b}$ (Hz\,s$^{-1}$)",
          "$\ddot{\\nu}_\mathrm{b}$ (Hz\,s$^{-2}$)",
          "$\dot{x}$ (s\,s$^{-1}$)",
          "$P_\mathrm{b}$ (days)",
          "$T_0$ (MJD)",
          "$\omega$ (\\arcdeg)",
          "$e$",
          "$f_\mathrm{M} (\Msun)",
          "$M_\mathrm{c} (\Msun)"]

fbs = []  
a1s = []  
tascs = []
eps1s = []
eps2s = []
fb1s = [] 
fb2s = [] 
xdots = []
pbs = []  
t0s = []  
oms = []  
eccs = [] 
fms = []  
mcs = []

binary_psrs = ['J0032+6946','J0214+5222','J0636+5128','J1239+3239','J1816+4510']
for bp in binary_psrs:
    par_path = f"data/{bp}_fiore+23.par"
    
    fb1_tex = '\ldots'
    fb2_tex = '\ldots'
    xdot_tex = '\ldots'
    
    psr_tex = bp.replace('-','$-$')
    with open(par_path, 'r') as infile:
        #pbdot_tex = "\ldots"
        for l in infile.readlines():
            if l.startswith("A1 "):
                a1_val = float(l.split()[1])
                a1_err = float(l.split()[3])
                a1_tex = tu.ufve(a1_val,a1_err)
#             elif l.startswith("PB "):
#                 pb_val = float(l.split()[1])
#                 pb_err = float(l.split()[3])
#                 pb_tex = tu.ufve(pb_val,pb_err)
            elif l.startswith("FB0"):
                fb_val = float(l.split()[1])
                fb_err = float(l.split()[3])
                fb_tex = tu.ufve(fb_val,fb_err)
#             elif l.startswith("PBDOT"):
#                 pbdot_val = float(l.split()[1])
#                 pbdot_err = float(l.split()[3])
#                 pbdot_tex = tu.ufve(pbdot_val,pbdot_err)
            elif l.startswith("TASC"):
                tasc_val = float(l.split()[1])
                tasc_err = float(l.split()[3])
                tasc_tex = tu.ufve(tasc_val,tasc_err)
            elif l.startswith("EPS1 "):
                eps1_val = float(l.split()[1])
                eps1_err = float(l.split()[3])
                eps1_tex = tu.ufve(eps1_val,eps1_err)
            elif l.startswith("EPS2 "):
                eps2_val = float(l.split()[1])
                eps2_err = float(l.split()[3])
                eps2_tex = tu.ufve(eps2_val,eps2_err)
            elif l.startswith("FB1"):
                fb1_val = float(l.split()[1])
                fb1_err = float(l.split()[3])
                fb1_tex = tu.ufve(fb1_val, fb1_err)
            elif l.startswith("FB2"):
                fb2_val = float(l.split()[1])
                fb2_err = float(l.split()[3])
                fb2_tex = tu.ufve(fb2_val, fb2_err)
            elif l.startswith("XDOT") or l.startswith("A1DOT"):
                xdot_val = float(l.split()[1])
                try:
                    xdot_err = float(l.split()[3])
                    xdot_tex = tu.ufve(xdot_val, xdot_err)
                except IndexError:
                    xdot_tex = "\ldots"
            else:
                pass
    
    # Derive mass function and Mc,min
    pb = (1./(fb_val*u.Hz)).to(u.day).value
    pb_err = (fb_err*u.Hz/(fb_val*u.Hz)**2).to(u.day).value
    pb_tex = tu.ufve(pb,pb_err)
    ecc = np.sqrt(eps1_val**2+eps2_val**2)
    ecc_err = np.sqrt((eps1_val*eps1_err)**2+(eps2_val*eps2_err)**2)/ecc
    ecc_tex = tu.ufve(ecc,ecc_err)
    om = np.arctan2(eps1_val,eps2_val)*180./np.pi
    if om < 0.:
        om += 360.
    om_err = (180./np.pi)*np.sqrt((eps2_val*eps1_err)**2+(eps1_val*eps2_err)**2)/ecc**2
    if om_err < 1.0:
        om_tex = tu.ufve(om,om_err)
    else:
        om_tex = f"{int(round(om,0))}({int(round(om_err,0))})"
    t0 = tasc_val + pb*om/360.
    t0_err = np.sqrt(tasc_err**2+(pb_err*om/360.)**2+(pb*om_err/360.)**2)
    t0_tex = tu.ufve(t0,t0_err)
    fm = dq.mass_funct(pb*u.day,a1_val*u.lightsecond)
    fm_tex = f"{num2tex(fm.value):.2e}" # Msun
    mc = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
    mc_max = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=26.0*u.deg, mp=1.4 * u.solMass)
    mc_med = dq.companion_mass(pb*u.day,a1_val*u.lightsecond, i=60.0*u.deg, mp=1.4 * u.solMass)
    max_masses.append(mc_max)
    med_masses.append(mc_med)
    mc_tex = f"{mc.value:.3f}" # Msun

    fbs.append(fb_tex)
    a1s.append(a1_tex)
    tascs.append(tasc_tex)
    eps1s.append(eps1_tex)
    eps2s.append(eps2_tex)
    fb1s.append(fb1_tex)
    fb2s.append(fb2_tex)
    xdots.append(xdot_tex)
    pbs.append(pb_tex)
    t0s.append(t0_tex)
    oms.append(om_tex)
    eccs.append(ecc_tex)
    fms.append(fm_tex)
    mcs.append(mc_tex)

params = [param_names,fbs,a1s,tascs,eps1s,eps2s,fb1s,fb2s,xdots,pbs,t0s,oms,eccs,fms,mcs]

for i,param_name in enumerate(param_names):
    if i==0:
        print("\cutinhead{Measured}")
    if "(days)" in param_name:
        print("\cutinhead{Derived}")
    param_list = params[i+1]
    try:
        outstr = f"{param_name.split()[0]} & {param_name.split()[1]}"
    except IndexError:
        outstr = f"{param_name.split()[0]} &"
    for pm in param_list:
        outstr += f" & {pm}"
    outstr += " \\\\"
    print(outstr.replace(" \\times ","$\\times").replace(" -"," $-$").replace(" ^","$^").replace("} ","}$ "))

\cutinhead{Measured}
$\nu_\mathrm{b}$ & (Hz) & 2.193632768(6)$\times10^{-8}$ & 2.260385778(9)$\times10^{-8}$ & 0.0001739119636(3) & 2.833032117(4)$\times10^{-6}$ & 3.2070609894(7)$\times10^{-5}$ \\
$a\sin & i/c$ & 178.674768(3) & 174.565762(5) & 0.0089858(1) & 2.371127(2) & 0.5954006(6) \\
$T_\mathrm{asc}$ & (MJD) & 56399.134959(4) & 56339.115121(4) & 56711.9950666(4) & 57730.0800015(9) & 56945.0911546(2) \\
$e\sin\omega$ & & 0.00028554(2) & $-$0.00271145(5) & 1(2)$\times10^{-5}$ & 5(2)$\times10^{-6}$ & 8(2)$\times10^{-6}$ \\
$e\cos\omega$ & & $-$0.00044865(3) & $-$0.00458640(6) & $-$1(2)$\times10^{-5}$ & 0(2)$\times10^{-6}$ & $-$1(2)$\times10^{-6}$ \\
$\dot{\nu}_\mathrm{b}$ & (Hz\,s$^{-1}$) & \ldots & \ldots & $-$7.1(7)$\times10^{-20}$ & \ldots & \ldots \\
$\ddot{\nu}_\mathrm{b}$ & (Hz\,s$^{-2}$) & \ldots & \ldots & $-$2.4(8)$\times10^{-27}$ & \ldots & \ldots \\
$\dot{x}$ & (s\,s$^{-1}$) & \ldots & $-$2.1(6)$\times10^{-13}$ & \ldots & \ldots & \ldots \\
\cutinhead{Derived}
$P_\mathrm{

In [10]:
print([round(medmass.value,4) for medmass in med_masses])
print([round(maxmass.value,4) for maxmass in max_masses])

[0.4956, 0.4936, 0.0081, 0.1467, 0.1891]
[1.2125, 1.2067, 0.0161, 0.3099, 0.4071]


In [11]:
# # Auto-populate BT binary table
# binary_psrs_dd = ['J0032+6946','J0214+5222']

# for bp in binary_psrs_dd:
#     par_path = f"data/{bp}_fiore+23.par"
    
#     psr_tex = bp.replace('-','$-$')
#     with open(par_path, 'r') as infile:
#         for l in infile.readlines():
#             if l.startswith("A1"):
#                 a1_val = float(l.split()[1])
#                 a1_err = float(l.split()[3])
#                 a1_tex = tu.ufve(a1_val,a1_err)
#             elif l.startswith("PB "):
#                 pb_val = float(l.split()[1])
#                 pb_err = float(l.split()[3])
#                 pb_tex = tu.ufve(pb_val,pb_err)
#             elif l.startswith("T0"):
#                 t0_val = float(l.split()[1])
#                 t0_err = float(l.split()[3])
#                 t0_tex = tu.ufve(t0_val,t0_err)
#             elif l.startswith("ECC"):
#                 ecc_val = float(l.split()[1])
#                 ecc_err = float(l.split()[3])
#                 ecc_tex = tu.ufve(ecc_val,ecc_err)
#             elif l.startswith("OM"):
#                 om_val = float(l.split()[1])
#                 om_err = float(l.split()[3])
#                 om_tex = tu.ufve(om_val,om_err)
#             else:
#                 pass

#     # Derive mass function and Mc,min
#     fm = dq.mass_funct(pb_val*u.day,a1_val*u.lightsecond)
#     fm_tex = f"{num2tex(fm.value):.4e}" # Msun
#     mc = dq.companion_mass(pb_val*u.day,a1_val*u.lightsecond, i=90.0*u.deg, mp=1.4 * u.solMass)
#     mc_tex = f"{mc.value:.2f}" # Msun
    
#     outstr = f"{psr_tex} & {pb_tex} & {a1_tex} & {t0_tex} & {ecc_tex} & {om_tex} & {fm_tex} & {mc_tex} \\\\"
#     print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

In [12]:
# Auto-populate miscellaneous parameters table
for psr in psrs:    
    include=False
    par_path = f"data/{psr}_fiore+23.par"
    
    #f2_tex = '\ldots'
    px_tex = '\ldots'
    fb0_tex = '\ldots'
    fb1_tex = '\ldots'
    fb2_tex = '\ldots'
    xdot_tex = '\ldots'
    fd1_tex = '\ldots'
    
    with open(par_path, 'r') as infile:
        for l in infile.readlines():
#             if l.startswith("F2"):
#                 f2_val = float(l.split()[1])
#                 f2_err = float(l.split()[3])
#                 f2_tex = tu.ufve(f2_val, f2_err)
#                 include = True
            if l.startswith("PX"):
                px_val = float(l.split()[1])
                try:
                    px_err = float(l.split()[3])
                except:
                    px_val = 1.74816089886028 # J0636 has PX set to zero
                    px_err = 1.6203
                px_tex = tu.ufve(px_val, px_err)
                include = True
            elif l.startswith("FB0"):
                fb0_val = float(l.split()[1])
                fb0_err = float(l.split()[3])
                fb0_tex = tu.ufve(fb0_val, fb0_err)
                include = True
            elif l.startswith("FB1"):
                fb1_val = float(l.split()[1])
                fb1_err = float(l.split()[3])
                fb1_tex = tu.ufve(fb1_val, fb1_err)
                include = True
            elif l.startswith("FB2"):
                fb2_val = float(l.split()[1])
                fb2_err = float(l.split()[3])
                fb2_tex = tu.ufve(fb2_val, fb2_err)
                include = True
            elif l.startswith("XDOT"):
                xdot_val = float(l.split()[1])
                xdot_err = float(l.split()[3])
                xdot_tex = tu.ufve(xdot_val, xdot_err)
                include = True
            elif l.startswith("FD1"):
                fd1_val = float(l.split()[1])
                fd1_err = float(l.split()[3])
                fd1_tex = tu.ufve(fd1_val, fd1_err)
                include = True
                
    if include==True:
        psr_tex = psr.replace('-','$-$')

        outstr = f"{psr_tex} & {fb0_tex} & {fb1_tex} & {fb2_tex} & {fd1_tex} & {px_tex} \\\\"
        print(outstr.replace(" \\times ","$\\times$").replace(" -"," $-$").replace("^","$^").replace("}","}$"))

IndexError: list index out of range